In [ ]:
#hide

#default_exp core
from nbdev.showdoc import show_doc

In [ ]:
#export
from bs4 import BeautifulSoup
import requests
import lxml
import time
import json

# Get & Process All American data from Wikipedia

In [ ]:
#export
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
years = ['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
#HAVE TO MANUALLY GRAB 2009

In [ ]:
all_players = []
for y in years: 
    url = 'https://en.wikipedia.org/wiki/{}_College_Football_All-America_Team'.format(y)
    r = requests.get(url, headers=headers)
    aaSoup = BeautifulSoup(r.text, 'lxml')
    for x in aaSoup.find_all("li", class_=""):
        player = []
        if ((",") in x.text and ("(") in x.text and (")") in x.text and ("{") not in x.text \
            and ("Archived") not in x.text and ("Gridiron") not in x.text and ("edited") not in x.text \
            and ("all-purpose") not in x.text and ("Football") not in x.text):
            playerInfo = x.text.split(",", 1)
            #year
            player.append(y)
            #Name
            player.append(playerInfo[0])
            playerInfo[1] = playerInfo[1].replace("(Fla.)", "FL")
            playerLocationAwards = playerInfo[1].split("(",1)
            #School
            if ("CONSENSUS" in x.text):
                playerLocationAwards[0] = playerLocationAwards[0].replace("-- CONSENSUS --", "")
                player.append(playerLocationAwards[0])
            elif ("UNANIMOUS" in x.text):
                playerLocationAwards[0] = playerLocationAwards[0].replace("-- UNANIMOUS --", "")
                player.append(playerLocationAwards[0])
            else:
                player.append(playerLocationAwards[0].strip())
            #Awards String
            awardString = playerLocationAwards[1]
            #coaches (AFCA)
            if ("AFCA" in awardString):
                player.append("1")
            else:
                player.append("0")
            #associated press (AP)
            if ("AP" in awardString):
                player.append("1")
            else:
                player.append("0")
            #writers (FWAA)
            if ("FWAA" in awardString):
                player.append("1")
            else:
                player.append("0")
            #sporting news (TSN)
            if ("TSN" in awardString):
                player.append("1")
            else:
                player.append("0")
            #walter camp (WCFF)
            if ("WCFF" in awardString):
                player.append("1")
            else:
                player.append("0")
            all_players.append(player)
    print(all_players)
    time.sleep(5)
print(all_players)        

In [ ]:
final_aaSelections = []
aa_keys = ['year', 'player', 'school', 'afca', 'ap', 'fwaa', 'tsn', 'wcff']

for list in all_players:
    newdict = {aa_keys[i]: list[i] for i in range(len(aa_keys))}
    final_aaSelections.append(newdict)
    newdict = {}
print(final_aaSelections)

In [ ]:
with open("..//scraped_json//aaSelections.json", "w") as write_file:
    json.dump(final_aaSelections, write_file)